# Pandas workbook

In this notebook we will review some Pandas tricks. Examples will use the iris dataset from R.A. Fisher, 1936 as provided by the [UCI ML repo](https://archive.ics.uci.edu/dataset/53/iris).

In [ ]:
import pandas as pd
pd.options.mode.copy_on_write = True
import numpy as np

In [ ]:
# load iris dataset
iris = pd.read_csv('iris.csv')
iris

# Filtering

Now let's filter for the species being versicolor

In [ ]:
iris[iris['species'] == 'versicolor'].info()

In [ ]:
iris[iris['species'].isin(['versicolor', 'setosa'])].info()

Exercise: How many observations have sepal length in the upper 50% quantile and petal width greater than 2?

In [ ]:
thresh = np.quantile(iris['sepal_length'], 0.5)
len(iris[(iris['sepal_length'] > thresh) & (iris['petal_width']>2)])

# Selecting

We may want to select only a subset of columns

In [ ]:
iris[['sepal_length', 'species']]

In [ ]:
# select all columns except sepal length and species
iris[iris.columns.difference(['sepal_length', 'species'])]

Exercise: How would you select only the columns containing the word "length", without explicitly writing them out? (in a bigger dataset where there may be dozens of columns containing "length", you wouldn't want to type them all out).

In [ ]:
print(iris.columns.str.contains('length'))
print(iris.columns[iris.columns.str.contains('length')])
iris[iris.columns[iris.columns.str.contains('length')]]

We can also select columns of only certain types

In [ ]:
# select only numeric columns
iris.select_dtypes(include='number')

Let's make a new dataframe with a new column `sepal_sum`, which holds the sum of `sepal_length` and `sepal_width` for `versicolor` flowers.

In [ ]:
iris_vc = iris[iris['species'] == 'versicolor']
iris_vc['sepal_sum'] = iris_vc['sepal_length'] + iris_vc['sepal_width']
iris_vc.head()

In [ ]:
# multiply every column containing "sepal" by 2
iris_new = iris.copy()
iris_new.loc[:, iris_new.columns.str.contains('sepal')] *= 2
iris_new.head()

# `groupby`

In [ ]:
iris.groupby('species').mean()

In [ ]:
iris.groupby('species').apply(lambda x: x.max() - x.min())

Exercise: For each species, randomly select half of the observations and compute the 0.25 quantile for each feature (i.e., sepal_length, sepal_width, petal_length, and petal_width). Hint: see `DataFrame.sample()`

In [ ]:
iris.groupby('species').sample(25).groupby('species').quantile(0.25)

# `sort_values`

In [ ]:
iris.sort_values('petal_length')

In [ ]:
iris.sort_values(['petal_length', 'sepal_width'], ascending=[False, True])

Exercise: For each species, only keep the observations with the largest 10 sepal lengths. Then, sort the rows in order of decreasing sepal length. Hint: can you do this using an `apply` with a `group_by`?

In [ ]:
iris.groupby('species').apply(lambda x: x.sort_values('sepal_length', ascending=False).head(10), include_groups=False)
iris.groupby('species').apply(lambda x: x.nlargest(10, 'sepal_length'), include_groups=False)

# Merges (joins)

Let's make a couple dataframes to play around with merges (aka joins)

In [ ]:
import string
lowercase_data = pd.DataFrame(dict(id = range(2,7), lower = list(string.ascii_lowercase[1:6])))
lowercase_data

In [ ]:
uppercase_data = pd.DataFrame(dict(id = range(1,6), upper = list(string.ascii_uppercase[0:5])))
uppercase_data

We can use these to demonstrate all four kinds of joins. In each case, the DataFrames are automatically joined on the common column `id`

In [ ]:
pd.merge(lowercase_data, uppercase_data, how='inner')

In [ ]:
pd.merge(lowercase_data, uppercase_data, how='left')

In [ ]:
pd.merge(lowercase_data, uppercase_data, how='right')

In [ ]:
pd.merge(lowercase_data, uppercase_data, how='outer')

# Data cleaning

Lat's make a DataFrame which is rather messy.

In [ ]:
iris_messy = pd.concat((iris, iris.sample(n=50, replace=True))).sample(frac=1).reset_index(drop=True)
iris_messy.loc[np.random.sample(iris_messy.shape[0]) < 0.1,'species'] = pd.NA
iris_messy

Exercise: clean up `iris_messy` to remove duplicate rows and rows with an NA value in `species`.

In [ ]:
# clean up iris_messy
iris_messy = iris_messy.drop_duplicates()
iris_messy = iris_messy.dropna(subset=['species'])
iris_messy